In [1]:
# Install dependencies
!pip uninstall -y tensorflow
!pip install transformers

Found existing installation: tensorflow 2.8.0+zzzcolab20220506162203
Uninstalling tensorflow-2.8.0+zzzcolab20220506162203:
  Successfully uninstalled tensorflow-2.8.0+zzzcolab20220506162203
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 60.9 MB/s 
     |████████████████████████████████| 86 kB 6.6 MB/s 
     |████████████████████████████████| 596 kB 71.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.optim import lr_scheduler

import logging
logging.basicConfig(level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore")

In [4]:
class SentimentClassifier(nn.Module):
  """
  This class defines the model architecture which is simply a fully-connected
  layer on top of a pre-trained BERT model. 
  """

  def __init__(self, BERT_MODEL):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(BERT_MODEL)
    self.drop = nn.Dropout(p=0.1)
    self.out = nn.Linear(self.bert.config.hidden_size, 3) # Number of output classes = 3

  def forward(self, ids, mask, token_type_ids):
    last_hidden_state, pooled_output = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
    output = self.drop(pooled_output)
    return self.out(output)

In [5]:
class SentiHood:
  """
  This class tokenizes the input text using the pre-trained BERT tokenizer 
  (wordpiece) and returns the corresponding tensors.
  """

  def __init__(self, text, targets, tokenizer, max_len):
    self.text = text
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.targets = targets

  def __len__(self):
    return len(self.targets)

  def __getitem__(self, item):
    text = str(self.text[item])
    targets = self.targets[item]

    inputs = self.tokenizer.encode_plus(
        text,
        add_special_tokens = True,
        max_length = self.max_len,
        pad_to_max_length = True
    )

    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]
    token_type_ids = inputs["token_type_ids"]

    return {
        "ids": torch.tensor(ids, dtype=torch.long),
        "mask": torch.tensor(mask, dtype=torch.long),
        "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
        "targets": torch.tensor(targets, dtype=torch.long)
    }

In [6]:
def loss_function(outputs, targets):
	"""
	This function defines the loss function which is used to train the model, i.e.
	CrossEntropy.
	"""
	
	# probability, predicted = torch.max(outputs, 1)
	# print(f"Predicted = {predicted.cpu().detach().numpy()}\nTargets = {targets}")

	return nn.CrossEntropyLoss(reduction='mean')(outputs, targets)


In [7]:
def train_loop_function(data_loader, model, optimizer, device, location, aspect):
  """
  This function defines the training loop over the entire training set.
  """

  model.train()

  running_loss = 0.0
  for bi, d in enumerate(data_loader):
    ids = d["ids"]
    mask = d["mask"]
    token_type_ids = d["token_type_ids"]
    targets = d["targets"]

    ids = ids.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)
    token_type_ids = token_type_ids.to(device, dtype=torch.long)
    targets = targets.to(device, dtype=torch.long)

    optimizer.zero_grad()

    outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
    loss = loss_function(outputs, targets)

    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if bi % 10 == 0 and bi!=0:
      temp = f'Batch index = {bi}\tLoss = {running_loss/10}'
      print(temp)

      f1 = open('/content/drive/MyDrive/BERT-ABSA/Bert-single/LocationAspectModels/' + str(location) + str(aspect) + '/loss.txt', 'a+')
      temp = temp + '\n'
      f1.write(temp)
      f1.close()

      running_loss = 0.0

In [8]:
def eval_loop_function(data_loader, model, device, location, aspect):
  """
  This function defines the evaluation loop over the entire validation set.
  It also computes accuracy of the trained model, which is used to select the 
  best model.
  """

  model.eval()

  corrects = 0
  total = 0
  for bi, d in enumerate(data_loader):
    ids = d["ids"]
    mask = d["mask"]
    token_type_ids = d["token_type_ids"]
    targets = d["targets"]

    ids = ids.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)
    token_type_ids = token_type_ids.to(device, dtype=torch.long)
    targets = targets.to(device, dtype=torch.long)

    outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)

    _, predicted = torch.max(outputs, 1)
    total = total + targets.size(0)
    corrects = corrects + (predicted==targets).sum().item()

    print(f"bi: {bi}\tPredicted: {predicted}\tTargets: {targets}")

  accuracy = corrects / total * 100
  f1 = open('/content/drive/MyDrive/BERT-ABSA/Bert-single/LocationAspectModels/' + str(location) + str(aspect) + '/accuracy.txt', 'a+')
  temp = f"Corrects: {corrects}\tTotal: {total}\tAccuracy: {accuracy}\n"
  f1.write(temp)
  f1.close()

  return accuracy

In [9]:
def run():
  """
  This function defines hyperparameters, model and optimizer, loads required
  datasets and initiate the training and validation procedures.
  """

  TRAIN_MAX_LEN = 140
  VALID_MAX_LEN = 140
  TRAIN_BATCH_SIZE = 24
  VALID_BATCH_SIZE = 24
  EPOCHS = 1
  BERT_MODEL = 'bert-base-uncased'
  LEARNING_RATE = 2e-5

  locations = ['LOCATION1', 'LOCATION2']
  aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety','shopping', 'touristy', 'transit-location']
 
  # Train and validate a model for every `location-aspect` using corresponding 
  # datasets.

  for location in locations:
    for aspect in aspects:
      print(f"Starting {location} {aspect}...")
      
      training_set_path = '/content/drive/MyDrive/BERT-ABSA/Bert-single/TrainingData/' + str(location) + str(aspect) + '.csv'
      validation_set_path = '/content/drive/MyDrive/BERT-ABSA/Bert-single/ValidationData/' + str(location) + str(aspect) + '.csv'

      df_train = pd.read_csv(training_set_path)
      df_valid = pd.read_csv(validation_set_path)
      sentiment_mapping = {
          'Positive': 0,
          'Negative': 1,
          'None': 2
      }
      df_train['sentiment'] = df_train['sentiment'].map(sentiment_mapping)
      df_valid['sentiment'] = df_valid['sentiment'].map(sentiment_mapping)
      df_train = df_train.reset_index(drop=True)
      df_valid = df_valid.reset_index(drop=True)

      tokenizer = transformers.BertTokenizer.from_pretrained(BERT_MODEL)

      train_dataset = SentiHood(
          text = df_train['text'].values,
          targets = df_train['sentiment'].values,
          tokenizer = tokenizer,
          max_len = TRAIN_MAX_LEN
      )
      print(f"Training Set: {len(train_dataset)}")

      # Custom sampler to compensate class imbalance in the dataset
      # ========================================================================
      class_counts = []
      for i in range(3):
        class_counts.append(df_train[df_train['sentiment']==i].shape[0])
      print(f"Class Counts: {class_counts}")
      
      num_samples = sum(class_counts)
      labels = df_train['sentiment'].values

      class_weights = []
      for i in range(len(class_counts)):
        if class_counts[i] != 0:
          class_weights.append(num_samples/class_counts[i])
        else:
          class_weights.append(0)

      weights = [class_weights[labels[i]] for i in range(int(num_samples))]
      sampler = torch.utils.data.sampler.WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples))
      # ========================================================================

      train_data_loader = torch.utils.data.DataLoader(
          train_dataset,
          batch_size = TRAIN_BATCH_SIZE,
          shuffle = False,
          sampler = sampler
      )

      valid_dataset = SentiHood(
          text = df_valid['text'].values,
          targets = df_valid['sentiment'].values,
          tokenizer = tokenizer,
          max_len = VALID_MAX_LEN
      )
      print(f"Validation Set: {len(valid_dataset)}")

      valid_data_loader = torch.utils.data.DataLoader(
          valid_dataset,
          batch_size = VALID_BATCH_SIZE,
          shuffle = False
      )

      device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
      print(f"Device: {device}")

      model = SentimentClassifier(BERT_MODEL)
      model = model.to(device)

      num_train_steps = int(len(train_dataset) / TRAIN_BATCH_SIZE * EPOCHS)
      optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

      scheduler = lr_scheduler.StepLR(
          optimizer,
          step_size = 1,
          gamma = 0.8
      )

      for epoch in range(EPOCHS):
        train_loop_function(data_loader=train_data_loader, model=model, optimizer=optimizer, device=device, location=location, aspect=aspect)
        accuracy = eval_loop_function(data_loader=valid_data_loader, model=model, device=device, location=location, aspect=aspect)

        print(f"\nEpoch = {epoch}\tAccuracy Score = {accuracy}")
        print(f"Learning Rate = {scheduler.get_lr()[0]}\n")

        scheduler.step()

        torch.save(model, '/content/drive/MyDrive/BERT-ABSA/Bert-single/LocationAspectModels/' + str(location) + str(aspect) + '/'+ str(epoch) + '.bin')

if __name__ == "__main__":
  run()

Starting LOCATION1 dining...


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Training Set: 2977
Class Counts: [79, 1, 2897]
Validation Set: 747
Device: cuda:0


...

Batch index = 10	Loss = 1.0182258665561676
Batch index = 20	Loss = 0.5593830078840256
Batch index = 30	Loss = 0.29096187800168993
bi: 0	Predicted: tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       device='cuda:0')	Targets: tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       device='cuda:0')
bi: 1	Predicted: tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       device='cuda:0')	Targets: tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       device='cuda:0')
bi: 2	Predicted: tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       device='cuda:0')	Targets: tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       device='cuda:0')
bi: 3	Predicted: tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2],
       device='cuda:0')	Targets: tensor([2, 2, 2, 2, 2, 2, 2, 2,

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max 

Batch index = 10	Loss = 0.938890254497528
Batch index = 20	Loss = 0.4740589320659637
Batch index = 30	Loss = 0.29540554583072665
bi: 0	Predicted: tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       device='cuda:0')	Targets: tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       device='cuda:0')
bi: 1	Predicted: tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       device='cuda:0')	Targets: tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       device='cuda:0')
bi: 2	Predicted: tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       device='cuda:0')	Targets: tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       device='cuda:0')
bi: 3	Predicted: tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       device='cuda:0')	Targets: tensor([2, 2, 2, 2, 2, 2, 2, 2, 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max 

Batch index = 10	Loss = 1.1737170219421387
Batch index = 20	Loss = 0.934897243976593
Batch index = 30	Loss = 0.7539389610290528
bi: 0	Predicted: tensor([2, 2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 1, 2, 2, 2],
       device='cuda:0')	Targets: tensor([2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2],
       device='cuda:0')
bi: 1	Predicted: tensor([0, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0],
       device='cuda:0')	Targets: tensor([2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       device='cuda:0')
bi: 2	Predicted: tensor([2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 0, 2, 2, 2, 2, 0, 2],
       device='cuda:0')	Targets: tensor([2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       device='cuda:0')
bi: 3	Predicted: tensor([0, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 2],
       device='cuda:0')	Targets: tensor([0, 0, 2, 2, 2, 2, 2, 2, 2